In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from tqdm import tqdm
from glob import glob
import gc
import cv2
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import math
import zipfile

In [2]:
! pip install tensorflow==2.4.0 tensorflow-gpu==2.4.0

^C


### Data

In [3]:
path = 'F:/Dacon/data/'
train_csv = pd.read_csv('F:/Dacon/data/train.csv')
test_csv = pd.read_csv('F:/Dacon/data/test.csv')

In [4]:
display(train_csv)
display(test_csv)

,img_id,input_img,label_img
0,10000,train_input_10000.png,train_label_10000.png
1,10001,train_input_10001.png,train_label_10001.png
2,10002,train_input_10002.png,train_label_10002.png
3,10003,train_input_10003.png,train_label_10003.png
4,10004,train_input_10004.png,train_label_10004.png
...,...,...,...
617,10617,train_input_10617.png,train_label_10617.png
618,10618,train_input_10618.png,train_label_10618.png
619,10619,train_input_10619.png,train_label_10619.png
620,10620,train_input_10620.png,train_label_10620.png


,img_id,input_img,submission_name
0,20000,test_input_20000.png,test_20000.png
1,20001,test_input_20001.png,test_20001.png
2,20002,test_input_20002.png,test_20002.png
3,20003,test_input_20003.png,test_20003.png
4,20004,test_input_20004.png,test_20004.png
5,20005,test_input_20005.png,test_20005.png
6,20006,test_input_20006.png,test_20006.png
7,20007,test_input_20007.png,test_20007.png
8,20008,test_input_20008.png,test_20008.png
9,20009,test_input_20009.png,test_20009.png


In [4]:
x, y = path+'train_input_img/'+train_csv['input_img'], path+'train_label_img/'+train_csv['label_img']

In [6]:
x.shape, y.shape

((622,), (622,))

In [5]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3, random_state=42, shuffle=True)

In [8]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((435,), (435,), (187,), (187,))

In [9]:
con_train_x = train_x.to_numpy()
con_test_x = test_x.to_numpy()
con_train_y = train_y.to_numpy()
con_test_y = test_y.to_numpy()

In [6]:
for input_path, label_path in zip(con_train_x, con_train_y):
    inp_img = cv2.imread(input_path)
    targ_img = cv2.imread(label_path)
    plt.figure(figsize=(15,10))
    inp_img = cv2.cvtColor(inp_img, cv2.COLOR_BGR2RGB)
    targ_img = cv2.cvtColor(targ_img, cv2.COLOR_BGR2RGB)
    plt.subplot(1,2,1)
    plt.imshow(inp_img)
    plt.subplot(1,2,2)
    plt.imshow(targ_img)
    plt.show()
    print(input_path, label_path, '\n')
    break

NameError: name 'con_train_x' is not defined

### HyperParameter

In [7]:
BATCH_SIZE = 8
img_size = 256
#weights = None
weights = 'imagenet'
learning_rate = 1e-5
EPOCHS = 5
dropout_rate = 0.1


### Preprocessing


In [8]:
def cut_img(img_path_list, save_path, stride):
    os.makedirs(f'{save_path}{img_size}', exist_ok=True)
    num = 0
    for path in tqdm(img_path_list):
        img = cv2.imread(path)
        for top in range(0, img.shape[0], stride):
            for left in range(0, img.shape[1], stride):
                piece = np.zeros([img_size, img_size, 3], np.uint8)
                temp = img[top:top+img_size, left:left+img_size, :]
                piece[:temp.shape[0], :temp.shape[1], :] = temp
                np.save(f'{save_path}{img_size}/{num}.npy', piece)
                num+=1


In [24]:
cut_img(train_x, path+'train_input_img/', 128)
cut_img(test_x, path+'train_input_img/', 128)
cut_img(train_y, path+'train_label_img/', 128)
cut_img(test_y, path+'train_label_img/', 128)

100%|██████████| 187/187 [00:48<00:00,  3.87it/s]


In [9]:
train_inp_files = glob(path+f'train_input_img/{img_size}/*.npy')
train_targ_files = glob(path+f'train_label_img/{img_size}/*.npy')

val_inp_files = glob(path+f'train_input_img/{img_size}/*.npy')
val_targ_files = glob(path+f'train_label_img/{img_size}/*.npy')

In [10]:
train_inp_files, train_targ_files = shuffle(train_inp_files, train_targ_files, random_state=42)


In [14]:
len(train_inp_files), len(val_inp_files), len(train_targ_files), len(val_targ_files)

(127920, 127920, 127920, 127920)

### Data_Set


In [11]:
def train_map_func(inp_path, targ_path):
    inp = np.load(inp_path)
    inp = inp.astype(np.float32)/255
    targ = np.load(targ_path)
    targ = targ.astype(np.float32)/255
    inp, targ = augmentation(inp, targ)

    return inp, targ

def val_map_func(inp_path, targ_path):
    inp = np.load(inp_path)
    inp = inp.astype(np.float32)/255
    targ = np.load(targ_path)
    targ = targ.astype(np.float32)/255
    return inp, targ

def augmentation(inp, targ):
    inp, targ = random_rot(inp, targ)
    inp, targ = random_flip(inp, targ)

    return inp, targ

def random_rot(inp, targ):
    k = np.random.randint(4)
    inp = np.rot90(inp, k)
    targ = np.rot90(targ, k)

    return inp, targ

def random_flip(inp, targ):
    f = np.random.randint(2)
    if f == 0:
        inp = np.fliplr(inp)
        targ = np.fliplr(targ)

    return inp, targ


In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_inp_files, train_targ_files))
train_dataset = train_dataset.map(lambda item1, item2: tf.numpy_function(train_map_func, [item1, item2], [tf.float32, tf.float32]), num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_inp_files, val_targ_files))
val_dataset = val_dataset.map(lambda item1, item2: tf.numpy_function(val_map_func, [item1, item2], [tf.float32, tf.float32]), num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


In [13]:
next(iter(train_dataset))[0].shape, next(iter(train_dataset))[1].shape, next(iter(val_dataset))[0].shape, next(iter(val_dataset))[1].shape

(TensorShape([8, 256, 256, 3]),
 TensorShape([8, 256, 256, 3]),
 TensorShape([8, 256, 256, 3]),
 TensorShape([8, 256, 256, 3]))

### Model

In [14]:
def convolution_block(x, filters, size, strides=(1,1), padding='same', activation=True):
    x = tf.keras.layers.Conv2D(filters, size, strides=strides, padding=padding)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if activation == True:
        x = tf.keras.layers.LeakyReLU(alpha=0.1)(x)
    return x

def residual_block(blockInput, num_filters=16):
    x = tf.keras.layers.LeakyReLU(alpha=0.1)(blockInput)
    x = tf.keras.layers.BatchNormalization()(x)
    blockInput = tf.keras.layers.BatchNormalization()(blockInput)
    x = convolution_block(x, num_filters, (3,3) )
    x = convolution_block(x, num_filters, (3,3), activation=False)
    x = tf.keras.layers.Add()([x, blockInput])
    return x


In [15]:
def ResUNet101V2(input_shape=(None, None, 3), dropout_rate=0.1, start_neurons = 16):
    backbone = tf.keras.applications.ResNet101V2(weights=weights, include_top=False, input_shape=input_shape)
    input_layer = backbone.input

    conv4 = backbone.layers[122].output
    conv4 = tf.keras.layers.LeakyReLU(alpha=0.1)(conv4)
    pool4 = tf.keras.layers.MaxPooling2D((2, 2))(conv4)
    pool4 = tf.keras.layers.Dropout(dropout_rate)(pool4)

    convm = tf.keras.layers.Conv2D(start_neurons * 32, (3, 3), activation=None, padding="same")(pool4)
    convm = residual_block(convm,start_neurons * 32)
    convm = residual_block(convm,start_neurons * 32)
    convm = tf.keras.layers.LeakyReLU(alpha=0.1)(convm)

    deconv4 = tf.keras.layers.Conv2DTranspose(start_neurons * 16, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = tf.keras.layers.concatenate([deconv4, conv4])
    uconv4 = tf.keras.layers.Dropout(dropout_rate)(uconv4)

    uconv4 = tf.keras.layers.Conv2D(start_neurons * 16, (3, 3), activation=None, padding="same")(uconv4)
    uconv4 = residual_block(uconv4,start_neurons * 16)
    uconv4 = residual_block(uconv4,start_neurons * 16)
    uconv4 = tf.keras.layers.LeakyReLU(alpha=0.1)(uconv4)

    deconv3 = tf.keras.layers.Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(uconv4)
    conv3 = backbone.layers[76].output
    uconv3 = tf.keras.layers.concatenate([deconv3, conv3])
    uconv3 = tf.keras.layers.Dropout(dropout_rate)(uconv3)

    uconv3 = tf.keras.layers.Conv2D(start_neurons * 8, (3, 3), activation=None, padding="same")(uconv3)
    uconv3 = residual_block(uconv3,start_neurons * 8)
    uconv3 = residual_block(uconv3,start_neurons * 8)
    uconv3 = tf.keras.layers.LeakyReLU(alpha=0.1)(uconv3)

    deconv2 = tf.keras.layers.Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv3)
    conv2 = backbone.layers[30].output
    uconv2 = tf.keras.layers.concatenate([deconv2, conv2])

    uconv2 = tf.keras.layers.Dropout(0.1)(uconv2)
    uconv2 = tf.keras.layers.Conv2D(start_neurons * 4, (3, 3), activation=None, padding="same")(uconv2)
    uconv2 = residual_block(uconv2,start_neurons * 4)
    uconv2 = residual_block(uconv2,start_neurons * 4)
    uconv2 = tf.keras.layers.LeakyReLU(alpha=0.1)(uconv2)

    deconv1 = tf.keras.layers.Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv2)
    conv1 = backbone.layers[2].output
    uconv1 = tf.keras.layers.concatenate([deconv1, conv1])

    uconv1 = tf.keras.layers.Dropout(0.1)(uconv1)
    uconv1 = tf.keras.layers.Conv2D(start_neurons * 2, (3, 3), activation=None, padding="same")(uconv1)
    uconv1 = residual_block(uconv1,start_neurons * 2)
    uconv1 = residual_block(uconv1,start_neurons * 2)
    uconv1 = tf.keras.layers.LeakyReLU(alpha=0.1)(uconv1)

    uconv0 = tf.keras.layers.Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv1)
    uconv0 = tf.keras.layers.Dropout(0.1)(uconv0)
    uconv0 = tf.keras.layers.Conv2D(start_neurons * 1, (3, 3), activation=None, padding="same")(uconv0)
    uconv0 = residual_block(uconv0,start_neurons * 1)
    uconv0 = residual_block(uconv0,start_neurons * 1)
    uconv0 = tf.keras.layers.LeakyReLU(alpha=0.1)(uconv0)

    uconv0 = tf.keras.layers.Dropout(dropout_rate/2)(uconv0)
    output_layer = tf.keras.layers.Conv2D(3, (1,1), padding="same", activation="sigmoid")(uconv0)

    model = tf.keras.models.Model(input_layer, output_layer)

    return model

### Optimize

In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate)
model = ResUNet101V2(input_shape=(img_size, img_size, 3),dropout_rate=dropout_rate)
model.compile(loss='mae', optimizer=optimizer)

### Callback

In [17]:
callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath = 'models/baseline_model.h5',
        monitor='val_loss',
        save_best_only=True
    )
]

In [ ]:
hist = model.fit(train_dataset, epochs=EPOCHS, validation_data=val_dataset, callbacks=callbacks_list)

### Result



In [1]:
plt.plot(hist.history["loss"], label='train_loss')
plt.plot(hist.history["val_loss"], label='val_loss')
plt.title('loss_plot')
plt.legend()
plt.show()

NameError: name 'plt' is not defined

# Instance Segmentation

### HR-Net_OCR_V2

- Instance Segmentation에서 SOTA를 달성
- ㅁㅈ얌쟝ㅈㅁ
- ㅇㅁ쟈ㅓ얌ㅈ
    - ㅁㅈ애ㅓㅁㅈ애
    - 얌저얒ㅁ

---
* Requeriment
    - Python >= 3.8
    - conda >= 4.10.5
    - torch >=1.8.1

* 추가설치
    - 암ㅈ아
        - ㅇㅈ


RuntimeError: CUDA error: an illegal memory access was
encountered CUDA kernel errors might be asynchronously
reported at some other API call,so the stacktrace below might be
incorrect. For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

- 한글설명(번역)
- 한글설명(왜 발생했는지)
- 어떻게 해결하면 되는지

### Solution
### 해결방안
- CUDA Drviver
    - 현재 cuda-drvier version -> 10.1
    - nvidia-driver -> 450.61.92
    - cudnn ->
- 조치방안
    - mmsegmentation -> version : mmcv, mmdet
    - CUda -> 11.1
    - nvidia-driver -> 465.61.59
    - cudnn ->
모델 테스트

In [3]:
from yolov4 import *

!python detect.py --verioan

python: can't open file 'detect.py': [Errno 2] No such file or directory
